# FILTER() + WHERE() + ISIN() + CONTAIN() + WHEN()

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (
    SparkSession.builder
    .appName("example-filter-isin")
    .getOrCreate()
)

In [ ]:

dept = [("Finance",10), ("Marketing",20), ("Sales",30), ("IT",40), ("X",40), ("Y",40), ("Z",40)]
deptColumns = ["dept_name", "dept_id"]

deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.show(truncate=False)

In [ ]:
#  The argument to the where method cannot be a string.
deptDF.where("col(dept_id) >= 30")

In [ ]:
deptDF.where(col('dept_id') >= 30).show()

The code block displayed below contains an error. The code block should return a DataFrame where all entries in column supplier contain the letter combination et in this order. Find the error.
>
Code block:
>
- `itemsDf.filter(column(‘supplier’).isin(‘et’))`

In [ ]:
data = [(1, 'Thick Coat for Walking in the Snow', 'Sports Company Inc.'),
        (2, 'Elegant Outdoors Summer Dress', 'YetiX'),
        (3, 'Outdoors Backpack', 'Sports Company Inc.')]

columns = ["itemId", "itemName", "supplier"]

itemsDf = spark.createDataFrame(data=data, schema=columns)

itemsDf.printSchema()
itemsDf.show()

In [ ]:
itemsDf[itemsDf.supplier.isin("Sports Company Inc.")].show()

In [ ]:
itemsDf.filter(col("supplier").contains("Company")).show()

Which of the following code blocks returns a new DataFrame with only columns predError and values of every second row of DataFrame transactionsDf?
>
Entire DataFrame transactionsDf:
>
- `+————-+———+—–+——-+———+—-+`
- `|transactionId|predError|value|storeId|productId| f|`
- `+————-+———+—–+——-+———+—-+`
- `| 1| 3| 4| 25| 1|null|`
- `| 2| 6| 7| 2| 2|null|`
- `| 3| 3| null| 25| 3|null|`
- `| 4| null| null| 3| 2|null|`
- `| 5| null| null| null| 2|null|`
- `| 6| 3| 2| 25| 2|null|`
- `+————-+———+—–+——-+———+—-+`
>
- `transactionsDf.filter(col("transactionId").isin([3,4,6])).select([predError, value])`
- `transactionsDf.select(col("transactionId").isin([3,4,6]), "predError", "value")`
- `transactionsDf.filter("transactionId" % 2 == 0).select("predError", "value")`
- `transactionsDf.filter(col("transactionId") % 2 == 0).select("predError", "value")`
- `transactionsDf.createOrReplaceTempView("transactionsDf")`
- `spark.sql("FROM transactionsDf SELECT predError, value WHERE transactionId % 2 = 0")`

In [2]:
from pyspark.sql.types import *

data = [(1, 3, 4, 25, 1, None),
        (2, 6, 7, 2, 2, None),
        (3, 3, None, 25, 3, None),
        (4, None, None, 3, 2, None),
        (5, None, None, None, 2, None),
        (6, 3, 2, 25, 2, None)]

schema = StructType([StructField('transactionId', IntegerType(), True),
                     StructField('predError', IntegerType(), True),
                     StructField('value', IntegerType(), True),
                     StructField('storeId', IntegerType(), True),
                     StructField('productId', IntegerType(), True),
                     StructField('f', IntegerType(), True)])

transactionsDf = spark.createDataFrame(data=data, schema=schema)
transactionsDf.show()

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|null|
|            2|        6|    7|      2|        2|null|
|            3|        3| null|     25|        3|null|
|            4|     null| null|      3|        2|null|
|            5|     null| null|   null|        2|null|
|            6|        3|    2|     25|        2|null|
+-------------+---------+-----+-------+---------+----+



In [ ]:
transactionsDf.filter((col("productId")==3) | (col("productId")<1)).show()

In [9]:
# NameError: name 'productId' is not defined
transactionsDf.filter(productId==3 or productId<1)

# ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not'
transactionsDf.filter((col("productId")==3) or (col("productId")<1))

# Py4JError: An error occurred while calling o43.or. 
# Trace: py4j.Py4JException: Method or([class java.lang.Integer]) does not exist
transactionsDf.filter(col("productId")==3 | col("productId")<1)

# SyntaxError: invalid syntax "or"
transactionsDf.where("productId"=3).or("productId"<1))



SyntaxError: invalid syntax (<ipython-input-9-fd3de26aeb47>, line 12)

# FILTER() + WHERE() + ISIN() + CONTAIN() + WHEN()

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (
    SparkSession.builder
    .appName("example-filter-isin")
    .getOrCreate()
)

In [ ]:

dept = [("Finance",10), ("Marketing",20), ("Sales",30), ("IT",40), ("X",40), ("Y",40), ("Z",40)]
deptColumns = ["dept_name", "dept_id"]

deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.show(truncate=False)

In [ ]:
#  The argument to the where method cannot be a string.
deptDF.where("col(dept_id) >= 30")

In [ ]:
deptDF.where(col('dept_id') >= 30).show()

The code block displayed below contains an error. The code block should return a DataFrame where all entries in column supplier contain the letter combination et in this order. Find the error.
>
Code block:
>
- `itemsDf.filter(column(‘supplier’).isin(‘et’))`

In [ ]:
data = [(1, 'Thick Coat for Walking in the Snow', 'Sports Company Inc.'),
        (2, 'Elegant Outdoors Summer Dress', 'YetiX'),
        (3, 'Outdoors Backpack', 'Sports Company Inc.')]

columns = ["itemId", "itemName", "supplier"]

itemsDf = spark.createDataFrame(data=data, schema=columns)

itemsDf.printSchema()
itemsDf.show()

In [ ]:
itemsDf[itemsDf.supplier.isin("Sports Company Inc.")].show()

In [ ]:
itemsDf.filter(col("supplier").contains("Company")).show()

Which of the following code blocks returns a new DataFrame with only columns predError and values of every second row of DataFrame transactionsDf?
>
Entire DataFrame transactionsDf:
>
- `+————-+———+—–+——-+———+—-+`
- `|transactionId|predError|value|storeId|productId| f|`
- `+————-+———+—–+——-+———+—-+`
- `| 1| 3| 4| 25| 1|null|`
- `| 2| 6| 7| 2| 2|null|`
- `| 3| 3| null| 25| 3|null|`
- `| 4| null| null| 3| 2|null|`
- `| 5| null| null| null| 2|null|`
- `| 6| 3| 2| 25| 2|null|`
- `+————-+———+—–+——-+———+—-+`
>
- `transactionsDf.filter(col("transactionId").isin([3,4,6])).select([predError, value])`
- `transactionsDf.select(col("transactionId").isin([3,4,6]), "predError", "value")`
- `transactionsDf.filter("transactionId" % 2 == 0).select("predError", "value")`
- `transactionsDf.filter(col("transactionId") % 2 == 0).select("predError", "value")`
- `transactionsDf.createOrReplaceTempView("transactionsDf")`
- `spark.sql("FROM transactionsDf SELECT predError, value WHERE transactionId % 2 = 0")`

In [ ]:
from pyspark.sql.types import *

data = [(1, 3, 4, 25, 1, None),
        (2, 6, 7, 2, 2, None),
        (3, 3, None, 25, 3, None),
        (4, None, None, 3, 2, None),
        (5, None, None, None, 2, None),
        (6, 3, 2, 25, 2, None)]

schema = StructType([StructField('transactionId', IntegerType(), True),
                     StructField('predError', IntegerType(), True),
                     StructField('value', IntegerType(), True),
                     StructField('storeId', IntegerType(), True),
                     StructField('productId', IntegerType(), True),
                     StructField('f', IntegerType(), True)])

transactionsDf = spark.createDataFrame(data=data, schema=schema)
transactionsDf.show()

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            1|        3|    4|     25|        1|null|
|            2|        6|    7|      2|        2|null|
|            3|        3| null|     25|        3|null|
|            4|     null| null|      3|        2|null|
|            5|     null| null|   null|        2|null|
|            6|        3|    2|     25|        2|null|
+-------------+---------+-----+-------+---------+----+



In [ ]:
# NameError: name 'productId' is not defined
transactionsDf.filter(productId==3 or productId<1)

# ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not'
transactionsDf.filter((col("productId")==3) or (col("productId")<1))

# Py4JError: An error occurred while calling o43.or. 
# Trace: py4j.Py4JException: Method or([class java.lang.Integer]) does not exist
transactionsDf.filter(col("productId")==3 | col("productId")<1)

# SyntaxError: invalid syntax "or"
transactionsDf.where("productId"=3).or("productId"<1))



NameError: name 'productId' is not defined

Py4JError: An error occurred while calling o43.or. Trace:
py4j.Py4JException: Method or([class java.lang.Integer]) does not exist
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:341)
	at py4j.reflection.ReflectionEngine.getMethod(ReflectionEngine.java:349)
	at py4j.Gateway.invoke(Gateway.java:286)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)



SyntaxError: invalid syntax (<ipython-input-6-1e9072805d47>, line 1)

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            3|        3| null|     25|        3|null|
+-------------+---------+-----+-------+---------+----+



In [ ]:
transactionsDf.select(col("transactionId").isin([3,4,6]), "predError", "value").show()


In [ ]:
transactionsDf.filter(col("transactionId") % 2 == 0).select("predError", "value").show()


In [ ]:
spark.sql("FROM transactionsDf SELECT predError, value WHERE transactionId % 2 = 0").show()

In [ ]:
# NameError: name 'predError' is not defined
# transactionsDf.filter(col("transactionId").isin([3,4,6])).select([predError, value])

# TypeError: not all arguments converted during string formatting
# transactionsDf.filter("transactionId" % 2 == 0).select("predError", "value")


In [ ]:
cond1 = col("productId").isin(1)
cond2 = col("productId").isin(2)
cond3 = col("productId").isin(3)

transactionsDf.withColumn("productGroup", when(cond1, "Grupo1")
								         .when(cond2, "Grupo2")
								         .when(cond3, "Grupo3")
								         .otherwise("Grupo4")) \
								  .show()

In [ ]:
df = spark.createDataFrame(
    [
        (1, 13), 
        (2, 23),
		(3, 10),
		(4, 17),
		(5, 18),
		(6, 21),
		(7, 9),
		(8, 26),
		(9, 28),
		
    ], ["id", "Idade"])

cond1 = col("Idade").isin(10, 11, 12)
cond2 = col("Idade").isin(13, 14, 15)
cond3 = col("Idade").isin(16, 17, 18)

df.withColumn("Idade_Grupo", when(cond1, "Grupo1")
                            .when(cond2, "Grupo2")
                            .when(cond3, "Grupo3")
                            .otherwise("Grupo4")) \
			 .show()    

Which of the following code blocks returns a one-column DataFrame of all values in column supplier of DataFrame itemsDf that do not contain the letter X? In the DataFrame, every value should only be listed once.
>
Sample of DataFrame itemsDf:
>
- `+——+——————–+——————–+——————-+`
- `|itemId| itemName| attributes| supplier|`
- `+——+——————–+——————–+——————-+`
- `| 1|Thick Coat for Wa…|[blue, winter, cozy]|Sports Company Inc.|`
- `| 2|Elegant Outdoors …|[red, summer, fre…| YetiX|`
- `| 3| Outdoors Backpack|[green, summer, t…|Sports Company Inc.|`
- `+——+——————–+——————–+——————-+`

In [ ]:
data = [(1, 'Thick Coat for Walking in the Snow', ['blue', 'winter', 'cozy'], 'Sports Company Inc.'),
        (2, 'Elegant Outdoors Summer Dress', ['red', 'summer'], 'YetiX'),
        (3, 'Outdoors Backpack', ['green', 'summer'], 'Sports Company Inc.')]

columns = ["itemId", "itemName", "attributes", "supplier"]

itemsDf = spark.createDataFrame(data=data, schema=columns)

itemsDf.printSchema()
itemsDf.show()

In [ ]:
itemsDf.select(~col('supplier').contains('X')).distinct().show()


In [ ]:
itemsDf.filter(~col('supplier').contains('X')).select('supplier').distinct().show()


In [ ]:
# NameError: name 'supplier' is not defined
itemsDf.filter(col(supplier).not_contains('X')).select(supplier).distinct().show()

# ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.
itemsDf.filter(not(col('supplier').contains('X'))).select('supplier').unique().show()

# SyntaxError: invalid syntax !
itemsDf.filter(!col('supplier').contains('X')).select(col('supplier')).unique().show()

+-------------+---------+-----+-------+---------+----+
|transactionId|predError|value|storeId|productId|   f|
+-------------+---------+-----+-------+---------+----+
|            3|        3| null|     25|        3|null|
+-------------+---------+-----+-------+---------+----+



In [ ]:
transactionsDf.select(col("transactionId").isin([3,4,6]), "predError", "value").show()


In [ ]:
transactionsDf.filter(col("transactionId") % 2 == 0).select("predError", "value").show()


In [ ]:
spark.sql("FROM transactionsDf SELECT predError, value WHERE transactionId % 2 = 0").show()

In [ ]:
# NameError: name 'predError' is not defined
# transactionsDf.filter(col("transactionId").isin([3,4,6])).select([predError, value])

# TypeError: not all arguments converted during string formatting
# transactionsDf.filter("transactionId" % 2 == 0).select("predError", "value")


In [ ]:
cond1 = col("productId").isin(1)
cond2 = col("productId").isin(2)
cond3 = col("productId").isin(3)

transactionsDf.withColumn("productGroup", when(cond1, "Grupo1")
								         .when(cond2, "Grupo2")
								         .when(cond3, "Grupo3")
								         .otherwise("Grupo4")) \
								  .show()

In [ ]:
df = spark.createDataFrame(
    [
        (1, 13), 
        (2, 23),
		(3, 10),
		(4, 17),
		(5, 18),
		(6, 21),
		(7, 9),
		(8, 26),
		(9, 28),
		
    ], ["id", "Idade"])

cond1 = col("Idade").isin(10, 11, 12)
cond2 = col("Idade").isin(13, 14, 15)
cond3 = col("Idade").isin(16, 17, 18)

df.withColumn("Idade_Grupo", when(cond1, "Grupo1")
                            .when(cond2, "Grupo2")
                            .when(cond3, "Grupo3")
                            .otherwise("Grupo4")) \
			 .show()    

Which of the following code blocks returns a one-column DataFrame of all values in column supplier of DataFrame itemsDf that do not contain the letter X? In the DataFrame, every value should only be listed once.
>
Sample of DataFrame itemsDf:
>
- `+——+——————–+——————–+——————-+`
- `|itemId| itemName| attributes| supplier|`
- `+——+——————–+——————–+——————-+`
- `| 1|Thick Coat for Wa…|[blue, winter, cozy]|Sports Company Inc.|`
- `| 2|Elegant Outdoors …|[red, summer, fre…| YetiX|`
- `| 3| Outdoors Backpack|[green, summer, t…|Sports Company Inc.|`
- `+——+——————–+——————–+——————-+`

In [ ]:
data = [(1, 'Thick Coat for Walking in the Snow', ['blue', 'winter', 'cozy'], 'Sports Company Inc.'),
        (2, 'Elegant Outdoors Summer Dress', ['red', 'summer'], 'YetiX'),
        (3, 'Outdoors Backpack', ['green', 'summer'], 'Sports Company Inc.')]

columns = ["itemId", "itemName", "attributes", "supplier"]

itemsDf = spark.createDataFrame(data=data, schema=columns)

itemsDf.printSchema()
itemsDf.show()

In [ ]:
itemsDf.select(~col('supplier').contains('X')).distinct().show()


In [ ]:
itemsDf.filter(~col('supplier').contains('X')).select('supplier').distinct().show()


In [ ]:
# NameError: name 'supplier' is not defined
itemsDf.filter(col(supplier).not_contains('X')).select(supplier).distinct().show()

# ValueError: Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.
itemsDf.filter(not(col('supplier').contains('X'))).select('supplier').unique().show()

# SyntaxError: invalid syntax !
itemsDf.filter(!col('supplier').contains('X')).select(col('supplier')).unique().show()